# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests, math, xlsxwriter
from scipy.stats import percentileofscore as score #Good for calculating percentile score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data =requests.get(api_url).json()
data


{'companyName': 'Apple Inc',
 'marketcap': 2244005307799,
 'week52high': 134.67,
 'week52low': 57.82,
 'week52change': 0.867245241655017,
 'sharesOutstanding': 17775768115,
 'float': 0,
 'avg10Volume': 103456048,
 'avg30Volume': 98378666,
 'day200MovingAvg': 113.71,
 'day50MovingAvg': 119.84,
 'employees': 0,
 'ttmEPS': 3.32,
 'ttmDividendRate': 0.8277431155498701,
 'dividendYield': 0.006423837219590002,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-26',
 'nextEarningsDate': '0',
 'peRatio': 38.58392225328285,
 'beta': 1.1794352175869083,
 'maxChangePercent': 49.992499719907045,
 'year5ChangePercent': 4.091685509361996,
 'year2ChangePercent': 2.181697017920193,
 'year1ChangePercent': 0.848128603599599,
 'ytdChangePercent': 0.7598205045196362,
 'month6ChangePercent': 0.476406450637098,
 'month3ChangePercent': 0.14713720895208127,
 'month1ChangePercent': 0.06419491692998527,
 'day30ChangePercent': 0.0629471739209233,
 'day5ChangePercent': 0.05080750648973272}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.848128603599599

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
   # print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_df = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data =requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(pd.Series([symbol,data[symbol]['price'],data[symbol]['stats']['year1ChangePercent'],'N/A'],index = my_columns),ignore_index=True)
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,120.152,0.405979,N/A
1,AAL,17.009,-0.410391,N/A
2,AAP,165.630,0.060205,N/A
3,AAPL,133.530,0.845448,N/A
4,ABBV,107.500,0.245910,N/A
...,...,...,...,...
95,CINF,83.658,-0.195695,N/A
96,CL,89.358,0.276587,N/A
97,CLX,207.610,0.386981,N/A
98,CMA,54.773,-0.211675,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_df.sort_values('One-Year Price Return',ascending = False, inplace =True)
final_df = final_df[:50]
final_df.reset_index(inplace = True)
final_df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,40.346,2.228131,N/A
1,31,AMD,101.082,1.312871,N/A
2,23,ALB,135.980,1.135727,N/A
3,24,ALGN,541.980,0.956780,N/A
4,3,AAPL,133.530,0.845448,N/A
5,85,CDNS,127.510,0.844092,N/A
6,36,AMZN,3382.900,0.834595,N/A
7,13,ADSK,307.000,0.597437,N/A
8,63,BIO,588.513,0.585004,N/A
9,6,ABMD,276.740,0.580568,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')
    
    try:
        float(portfolio_size)
    except:
        print('This is not a number, please try again.')
        portfolio_size = input('Enter the size of your portfolio: ')
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 1000000
1000000


In [9]:
position_size = float(portfolio_size)/len(final_df)
for i in range(0,len(final_df)):
    final_df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Price'])
    
final_df

C:\Users\q3604\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,40.346,2.228131,495
1,31,AMD,101.082,1.312871,197
2,23,ALB,135.980,1.135727,147
3,24,ALGN,541.980,0.956780,36
4,3,AAPL,133.530,0.845448,149
5,85,CDNS,127.510,0.844092,156
6,36,AMZN,3382.900,0.834595,5
7,13,ADSK,307.000,0.597437,65
8,63,BIO,588.513,0.585004,33
9,6,ABMD,276.740,0.580568,72


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = ['Ticker',
               'Price',
               'Number of Shares to Buy',
               'One-Year Price Return',
               'One-Year Return Percentile',
               'Six-Month Price Return',
               'Six-Month Return Percentile',
               'Three-Month Price Return',
               'Three-Month Return Percentile',
               'One-Month Price Return',
               'One-Month Return Percentile',
               'HQM Score'
              ]

hqm_df = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data =requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(pd.Series([
            symbol,
            data[symbol]['price'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
        ], index = hqm_columns),ignore_index = True)

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.120,N/A,0.406037,N/A,0.362171,N/A,0.176354,N/A,0.0656032,N/A,N/A
1,AAL,17.283,N/A,-0.408448,N/A,-0.0104647,N/A,0.227572,N/A,0.328899,N/A,N/A
2,AAP,168.260,N/A,0.0595311,N/A,0.136716,N/A,0.0470314,N/A,0.0692431,N/A,N/A
3,AAPL,130.500,N/A,0.884374,N/A,0.475428,N/A,0.143083,N/A,0.0637778,N/A,N/A
4,ABBV,106.690,N/A,0.236816,N/A,0.120624,N/A,0.184669,N/A,0.0648194,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.840,N/A,0.0890247,N/A,0.182526,N/A,0.143729,N/A,0.0308905,N/A,N/A
501,ZBH,154.280,N/A,0.0002256,N/A,0.141181,N/A,0.0228212,N/A,-0.00928831,N/A,N/A
502,ZBRA,393.746,N/A,0.489084,N/A,0.421071,N/A,0.49402,N/A,0.0921216,N/A,N/A
503,ZION,43.200,N/A,-0.153201,N/A,0.159251,N/A,0.341648,N/A,0.0540374,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]



for row in hqm_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile' 
        Return_col = hqm_df[change_col].dropna() #Some rows has None Object
        try:
            hqm_df.loc[row, percentile_col] = score(Return_col, hqm_df.loc[row,change_col])/100
        except:
            pass
        
hqm_df.dropna(inplace=True)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.120,N/A,0.406037,0.866267,0.362171,0.812375,0.176354,0.634731,0.0656032,0.740519,N/A
1,AAL,17.283,N/A,-0.408448,0.0259481,-0.0104647,0.121756,0.227572,0.776447,0.328899,0.992016,N/A
2,AAP,168.260,N/A,0.0595311,0.469062,0.136716,0.369261,0.0470314,0.279441,0.0692431,0.746507,N/A
3,AAPL,130.500,N/A,0.884374,0.982036,0.475428,0.904192,0.143083,0.546906,0.0637778,0.724551,N/A
4,ABBV,106.690,N/A,0.236816,0.728543,0.120624,0.335329,0.184669,0.656687,0.0648194,0.730539,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.840,N/A,0.0890247,0.516966,0.182526,0.487026,0.143729,0.550898,0.0308905,0.578842,N/A
501,ZBH,154.280,N/A,0.0002256,0.377246,0.141181,0.387226,0.0228212,0.219561,-0.00928831,0.347305,N/A
502,ZBRA,393.746,N/A,0.489084,0.908184,0.421071,0.868263,0.49402,0.972056,0.0921216,0.802395,N/A
503,ZION,43.200,N/A,-0.153201,0.163673,0.159251,0.413174,0.341648,0.908184,0.0540374,0.692615,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.120,N/A,0.406037,0.866267,0.362171,0.812375,0.176354,0.634731,0.0656032,0.740519,0.763473
1,AAL,17.283,N/A,-0.408448,0.0259481,-0.0104647,0.121756,0.227572,0.776447,0.328899,0.992016,0.479042
2,AAP,168.260,N/A,0.0595311,0.469062,0.136716,0.369261,0.0470314,0.279441,0.0692431,0.746507,0.466068
3,AAPL,130.500,N/A,0.884374,0.982036,0.475428,0.904192,0.143083,0.546906,0.0637778,0.724551,0.789421
4,ABBV,106.690,N/A,0.236816,0.728543,0.120624,0.335329,0.184669,0.656687,0.0648194,0.730539,0.612774
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.840,N/A,0.0890247,0.516966,0.182526,0.487026,0.143729,0.550898,0.0308905,0.578842,0.533433
501,ZBH,154.280,N/A,0.0002256,0.377246,0.141181,0.387226,0.0228212,0.219561,-0.00928831,0.347305,0.332834
502,ZBRA,393.746,N/A,0.489084,0.908184,0.421071,0.868263,0.49402,0.972056,0.0921216,0.802395,0.887725
503,ZION,43.200,N/A,-0.153201,0.163673,0.159251,0.413174,0.341648,0.908184,0.0540374,0.692615,0.544411


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_df.sort_values('HQM Score', ascending = False, inplace = True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(inplace=True, drop = True)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,25.830,N/A,0.917993,0.984032,1.3163,0.998004,0.466996,0.96008,0.165749,0.916168,0.964571
1,ALGN,521.680,N/A,0.942543,0.986028,0.977119,0.992016,0.580863,0.982036,0.130883,0.872255,0.958084
2,TWTR,56.020,N/A,0.749553,0.962076,0.569152,0.944112,0.372139,0.926148,0.264633,0.982036,0.953593
3,LRCX,505.430,N/A,0.777331,0.966068,0.648259,0.962076,0.624457,0.986028,0.131326,0.874251,0.947106
4,AMAT,91.200,N/A,0.507463,0.916168,0.539496,0.94012,0.593455,0.984032,0.199257,0.946108,0.946607
5,XLNX,154.660,N/A,0.610071,0.942116,0.657607,0.966068,0.50387,0.976048,0.159198,0.902196,0.946607
6,LB,38.460,N/A,1.20859,0.994012,1.43714,1,0.298317,0.868263,0.117333,0.850299,0.928144
7,AMD,96.940,N/A,1.33445,0.996008,0.792872,0.986028,0.267185,0.830339,0.157145,0.9002,0.928144
8,PYPL,239.420,N/A,1.13574,0.992016,0.432197,0.874251,0.276199,0.848303,0.207691,0.958084,0.918164
9,MU,73.870,N/A,0.392973,0.858283,0.435887,0.88024,0.468364,0.964072,0.188052,0.932136,0.908683


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [15]:
position_size = float(portfolio_size)/len(hqm_df)
for i in range(0,len(hqm_df)):
    hqm_df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/hqm_df.loc[i, 'Price'])
    
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,25.830,774,0.917993,0.984032,1.3163,0.998004,0.466996,0.96008,0.165749,0.916168,0.964571
1,ALGN,521.680,38,0.942543,0.986028,0.977119,0.992016,0.580863,0.982036,0.130883,0.872255,0.958084
2,TWTR,56.020,357,0.749553,0.962076,0.569152,0.944112,0.372139,0.926148,0.264633,0.982036,0.953593
3,LRCX,505.430,39,0.777331,0.966068,0.648259,0.962076,0.624457,0.986028,0.131326,0.874251,0.947106
4,AMAT,91.200,219,0.507463,0.916168,0.539496,0.94012,0.593455,0.984032,0.199257,0.946108,0.946607
5,XLNX,154.660,129,0.610071,0.942116,0.657607,0.966068,0.50387,0.976048,0.159198,0.902196,0.946607
6,LB,38.460,520,1.20859,0.994012,1.43714,1,0.298317,0.868263,0.117333,0.850299,0.928144
7,AMD,96.940,206,1.33445,0.996008,0.792872,0.986028,0.267185,0.830339,0.157145,0.9002,0.928144
8,PYPL,239.420,83,1.13574,0.992016,0.432197,0.874251,0.276199,0.848303,0.207691,0.958084,0.918164
9,MU,73.870,270,0.392973,0.858283,0.435887,0.88024,0.468364,0.964072,0.188052,0.932136,0.908683


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_df.to_excel(writer, 'momentum_strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
hqm_columns = ['Ticker',
               'Price',
               'Number of Shares to Buy',
               'One-Year Price Return',
               'One-Year Return Percentile',
               'Six-Month Price Return',
               'Six-Month Return Percentile',
               'Three-Month Price Return',
               'Three-Month Return Percentile',
               'One-Month Price Return',
               'One-Month Return Percentile',
               'HQM Score'
              ]


column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', float_template],
}

for column in column_formats.keys():
    writer.sheets['momentum_strategy'].set_column(f'{column}:{column}',18, column_formats[column][1])
    writer.sheets['momentum_strategy'].write(f'{column}1',column_formats[column][0],column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()